# Rede Neural Quântica (VQC) no Breast Cancer

Este notebook treina e avalia uma Rede Neural Quântica (implementada como um VQC - Variational Quantum Classifier) no dataset Breast Cancer, utilizando PCA com 5 features.

In [ ]:

import time
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report


from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes

from qiskit.primitives import StatevectorSampler
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit_machine_learning.optimizers import COBYLA

from sklearn.decomposition import PCA

import matplotlib.pyplot as plt


In [8]:

X_full, y = load_breast_cancer(return_X_y=True)
feature_names = load_breast_cancer().feature_names


X_full_df = pd.DataFrame(X_full, columns=feature_names)

print("--- Dataset Breast Cancer Carregado ---")
print("Formato original:", X_full_df.shape)

--- Dataset Breast Cancer Carregado ---
Formato original: (569, 30)


In [9]:

X_train_full, X_test_full, y_train, y_test = train_test_split(
    X_full_df, y, test_size=0.25, random_state=42, stratify=y
)

print("Dados divididos em treino e teste.")
print("Tamanho do treino:", X_train_full.shape)
print("Tamanho do teste:", X_test_full.shape)

Dados divididos em treino e teste.
Tamanho do treino: (426, 30)
Tamanho do teste: (143, 30)


In [10]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test_full)
print("Dados normalizados.")


Dados normalizados.


In [ ]:

pca_analise = PCA()
pca_analise.fit(X_train_scaled)

N_COMPONENTES = 5
pca = PCA(n_components=N_COMPONENTES)

X_train_pca = pca.fit_transform(X_train_scaled)

X_test_pca = pca.transform(X_test_scaled)

In [12]:



num_features = X_train_pca.shape[1]


feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
optimizer = COBYLA(maxiter=100)
sampler = StatevectorSampler()


# O VQC não tem um parâmetro 'random_state' como os modelos do Scikit-learn.
# A aleatoriedade vem do ponto de partida do otimizador.
# Para garantir que os resultados sejam sempre os mesmos a cada execução,
# críasse um ponto inicial fixo usando um gerador de números aleatórios com uma semente (seed) de 42.
# Isso torna o experimento 100% reprodutível.

num_parametros = ansatz.num_parameters
rng = np.random.default_rng(42)
initial_point = rng.random(num_parametros)


vqc_bc = VQC(
    sampler=sampler,
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=initial_point 
)

print("instância VQC criada. \niniciando treinamento")


start_time = time.time()
vqc_bc.fit(X_train_pca, y_train)
end_time = time.time()
print(f"Modelo VQC treinado em {end_time - start_time:.2f} segundos.")

vqc_predictions_bc = vqc_bc.predict(X_test_pca)

print("\n--- Relatório de Classificação para a QNN/VQC (Breast Cancer) ---")
print(classification_report(y_test, vqc_predictions_bc, target_names=load_breast_cancer().target_names))

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


instância VQC criada. 
iniciando treinamento
Modelo VQC treinado em 455.62 segundos.

--- Relatório de Classificação para a QNN/VQC (Breast Cancer) ---
              precision    recall  f1-score   support

   malignant       0.44      0.60      0.51        53
      benign       0.70      0.56      0.62        90

    accuracy                           0.57       143
   macro avg       0.57      0.58      0.57       143
weighted avg       0.61      0.57      0.58       143

